# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
import sys
key = "C:\\Users\\Kelly\\Desktop\\KEYS"
sys.path.append(key)
from GoogleAPI import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv('clean_city_data.csv')
city_df

,Unnamed: 0,City ID,City Name,Cloudiness,Wind Speed,Temperature,Feels Like,Min Temp,Max Temp,Barometric Pressure,Humidity,Sea Level,Ground Level,Latitude,Longitude
0,0,2685750,Östersund,99.0,100.00,2721.0,267.45,272.04,272.15,1013.0,100.0,NaN,NaN,NaN,NaN
1,1,3831208,Qaanaaq,100.0,59.00,26381.0,258.75,263.81,263.81,1010.0,81.0,1010.0,1010.0,86.349741,82.118957
2,2,5554428,Ketchikan,1.0,60.00,27415.0,269.83,274.15,274.15,1011.0,80.0,NaN,NaN,-52.936276,16.030378
3,3,2206939,Bluff,56.0,11.62,28537.0,280.11,285.37,285.37,1025.0,80.0,NaN,NaN,-32.749052,85.423768
4,4,1525798,Balkhash,0.0,70.00,25215.0,244.26,252.15,252.15,1037.0,83.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,558,1805518,Jining,0.0,6.00,27508.0,266.40,275.08,275.08,1031.0,69.0,1031.0,1026.0,NaN,NaN
559,559,1856434,Murakami,75.0,360.00,27915.0,270.62,279.15,279.15,1013.0,87.0,NaN,NaN,NaN,NaN
560,560,2171069,Colac,100.0,5.81,29935.0,298.82,297.04,300.93,1017.0,45.0,NaN,NaN,NaN,NaN
561,561,3698608,Celendín,100.0,118.00,28458.0,284.25,284.58,284.58,1016.0,92.0,1016.0,748.0,NaN,NaN


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=gkey)
city_df = city_df.dropna()
locations = city_df[["Latitude", "Longitude"]].astype(float)

weights = city_df["Humidity"].astype(float)


In [69]:
# fig = gmaps.figure()

# heat_layer = gmaps.heatmap_layer(locations, weights, 
#                                  dissipating=False, max_intensity=100,
#                                  point_radius = 4)

# fig.add_layer(heat_layer)

# fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
ideal_weather = city_df.loc[city_df['Humidity'] < 50]
ideal_weather = city_df.loc[city_df['Humidity'] > 30]
ideal_weather = city_df.loc[city_df['Feels Like'] < 1026.28]
ideal_weather = city_df.loc[city_df['Feels Like'] > 1005.76]
ideal_weather = city_df.loc[city_df['Max Temp'] < 296.5]
ideal_weather = city_df.loc[city_df['Min Temp'] > 291.5]
ideal_weather = city_df.loc[city_df['Wind Speed'] < 24]
ideal_weather = city_df.loc[city_df['Cloudiness'] > 40]
dropped_weather = ideal_weather.dropna(subset=['Humidity', 'Feels Like', 'Max Temp', 'Min Temp', 'Wind Speed', 'Cloudiness', 'Latitude', 'Longitude'])
hotel_checker = dropped_weather.sample(10)
hotel_checker

,Unnamed: 0,City ID,City Name,Cloudiness,Wind Speed,Temperature,Feels Like,Min Temp,Max Temp,Barometric Pressure,Humidity,Sea Level,Ground Level,Latitude,Longitude
451,451,2123814,Leningradskiy,100.0,102.0,26049.0,255.39,260.49,260.49,1031.0,95.0,1031.0,1030.0,-69.820904,13.774080
168,168,4031574,Provideniya,100.0,45.0,26387.0,252.17,263.87,263.87,1005.0,95.0,1005.0,993.0,72.491104,40.293504
234,234,3418910,Upernavik,84.0,32.0,26509.0,261.04,265.09,265.09,1001.0,85.0,1001.0,1001.0,73.950017,170.348385
473,473,86049,Jalu,71.0,223.0,29072.0,285.30,290.72,290.72,1007.0,39.0,1007.0,1002.0,-41.373924,17.169218
23,23,3466704,Castro,70.0,48.0,28964.0,290.29,289.64,289.64,1014.0,92.0,1014.0,903.0,-70.189202,108.339165
271,271,2125693,Evensk,100.0,352.0,25848.0,251.77,258.48,258.48,1017.0,86.0,1017.0,1010.0,-76.673095,154.728901
203,203,2171099,Codrington,100.0,11.0,30099.0,297.21,300.99,300.99,1015.0,39.0,1015.0,1014.0,1.471777,-148.377458
195,195,2126123,Chokurdakh,100.0,326.0,24285.0,236.77,242.85,242.85,1032.0,87.0,1032.0,1026.0,79.051437,114.780421
112,112,1507390,Dikson,100.0,11.0,25007.0,242.49,250.07,250.07,1006.0,91.0,1006.0,999.0,-46.880381,-121.747164
413,413,6620339,Karratha,83.0,311.0,30127.0,300.07,301.27,301.27,1006.0,46.0,1006.0,1003.0,60.937497,76.379333


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = hotel_checker.reindex(columns = ["City Name", "Latitude", "Longitude"])
hotel_df["Hotel Name"] = ""
hotel_df

,City Name,Latitude,Longitude,Hotel Name
451,Leningradskiy,-69.820904,13.774080,
168,Provideniya,72.491104,40.293504,
234,Upernavik,73.950017,170.348385,
473,Jalu,-41.373924,17.169218,
23,Castro,-70.189202,108.339165,
271,Evensk,-76.673095,154.728901,
203,Codrington,1.471777,-148.377458,
195,Chokurdakh,79.051437,114.780421,
112,Dikson,-46.880381,-121.747164,
413,Karratha,60.937497,76.379333,


In [ ]:
# find the closest restaurant of each type to coordinates


base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
i = 0
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    lat = hotel_df["Latitude"].iloc[i]
    long = hotel_df["Longitude"].iloc[i]
    coords = lat, long
    i = i + 1
    city = row['City Name']
    params = {
        "location": coords,  
        "radius": 5000,
        "keyword": "lodging",
        "key": gkey,
    }
#     assemble url and make API request
    print(f"Retrieving Results for hotels near {city}:")
    response = requests.get(base_url, params=params).json()
    print(params)
    print(response)
#     # extract results
#     results = response['results']
    
    
#     try:
#         print(f"Closest hotel is {results[0]['name']}.")
        
#         types_df.loc[index, 'name'] = results[0]['name']
#         types_df.loc[index, 'address'] = results[0]['vicinity']
#         types_df.loc[index, 'price_level'] = results[0]['price_level']
#         types_df.loc[index, 'rating'] = results[0]['rating']
        
#     except (KeyError, IndexError):
#         print("Missing field/result... skipping.")
        
#     print("------------")

In [95]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'City'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
